In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [2]:
with open("the-verdict.txt" , 'r') as file:
    data = file.read()
# data = input("Enter your prompt here: ")

In [3]:
print(len(data))

20479


In [4]:
split_text = re.split(r'([,.<>/!@#$%^&*():;]|--|\s)',data)

In [5]:
#This a for loop for separating the text. And below is list comprehension

# result = []
# for item in text:
#     if item.strip():
#         result.append(item)
    
# result

In [6]:

# This is a word based tokenization algorithm. This tokenize for each word.....

result = [item.strip() for item in split_text if item.strip()]
print(result)

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"The', 'height', 'of', 'his', 'glory"', '--', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '--', 'his', 'last', 'Chicago', 'sitter', '--', 'deploring', 'his', 'unaccountable', 'abdication', '.', '"Of', 'course', "it's", 'going', 'to', 'send', 'the', 'value', 'of', 'my', 'picture', "'way", 'up', ';', 'but', 'I', "don't", 'think', 'of', 'that', ',', 'Mr', '.', 'Rickham', '--', '

In [7]:
len(result)

4364

In [8]:
words = sorted(set(result))
print(len(words))

1209


In [9]:
token_id = {token:i for i,token in enumerate(words)}
print(token_id)

{'!': 0, '"': 1, '"Ah': 2, '"Be': 3, '"Begin': 4, '"By': 5, '"Come': 6, '"Destroyed': 7, '"Don\'t': 8, '"Gisburns"': 9, '"Grindles': 10, '"Hang': 11, '"Has': 12, '"How': 13, '"I': 14, '"I\'d': 15, '"If': 16, '"It': 17, '"It\'s': 18, '"Jack': 19, '"Money\'s': 20, '"Moon-dancers"': 21, '"Mr': 22, '"Mrs': 23, '"My': 24, '"Never': 25, '"Of': 26, '"Oh': 27, '"Once': 28, '"Only': 29, '"Or': 30, '"That': 31, '"The': 32, '"Then': 33, '"There': 34, '"This': 35, '"We': 36, '"Well': 37, '"What': 38, '"When': 39, '"Why': 40, '"Yes': 41, '"You': 42, '"but': 43, '"deadening': 44, '"dragged': 45, '"effects"': 46, '"interesting"': 47, '"lift': 48, '"obituary"': 49, '"strongest': 50, '"strongly"': 51, '"sweetly"': 52, "'": 53, "'Are": 54, "'It's": 55, "'coming'": 56, "'done'": 57, "'subject": 58, "'technique'": 59, "'way": 60, '(': 61, ')': 62, ',': 63, '--': 64, '.': 65, ':': 66, ';': 67, 'A': 68, 'Among': 69, 'And': 70, 'Arrt': 71, 'As': 72, 'At': 73, 'Burlington': 74, 'But': 75, 'By': 76, 'Carlo': 7

In [10]:
import importlib
import tiktoken

print("tik version = ", importlib.metadata.version("tiktoken"))

tik version =  0.12.0


In [11]:
tokenizer = tiktoken.get_encoding("gpt2")
# Byte Pair Encoding ---------- BPE is done using tiktoken library 

In [12]:
text = ("Helllo, do you like tea? <|endoftext|> In the sunlight terraces"
        "of someUnknownPlace.")

interger = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(interger)

[28254, 5439, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 19606, 8812, 2114, 1659, 617, 20035, 27271, 13]


In [13]:
strings = tokenizer.decode(interger)
print(strings)

Helllo, do you like tea? <|endoftext|> In the sunlight terracesof someUnknownPlace.


In [14]:
# input-target pair
with open("the-verdict.txt" , 'r') as file:
    data = file.read()

In [15]:
enc_text = tokenizer.encode(data)
print(len(enc_text))

5145


In [16]:
sample_context = 20
for i in range(1, sample_context+1):
    context = enc_text[:i]
    desire = enc_text[i]

    print(tokenizer.decode(context), "------->", tokenizer.decode([desire]))

I ------->  H
I H -------> AD
I HAD ------->  always
I HAD always ------->  thought
I HAD always thought ------->  Jack
I HAD always thought Jack ------->  G
I HAD always thought Jack G -------> is
I HAD always thought Jack Gis -------> burn
I HAD always thought Jack Gisburn ------->  rather
I HAD always thought Jack Gisburn rather ------->  a
I HAD always thought Jack Gisburn rather a ------->  cheap
I HAD always thought Jack Gisburn rather a cheap ------->  genius
I HAD always thought Jack Gisburn rather a cheap genius -------> --
I HAD always thought Jack Gisburn rather a cheap genius-- -------> though
I HAD always thought Jack Gisburn rather a cheap genius--though ------->  a
I HAD always thought Jack Gisburn rather a cheap genius--though a ------->  good
I HAD always thought Jack Gisburn rather a cheap genius--though a good ------->  fellow
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow ------->  enough
I HAD always thought Jack Gisburn rather a chea

In [17]:
# Creating a input output pair

from torch.utils.data import Dataset, DataLoader
import torch

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_len, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
    # Using sliding window to chuck block into overlapping sequence of max_len 
        for i in range (0, len(token_ids)-max_len, stride):
            input_chuck = token_ids[i : i + max_len]
            target_chuck = token_ids[i + 1 : i + max_len + 1]
            self.input_ids.append(torch.tensor(input_chuck))
            self.target_ids.append(torch.tensor(target_chuck))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    

def create_dataloader(txt, batch_size = 4, max_len = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    dataset = GPTDatasetV1(txt, tokenizer, max_len, stride) #create dataset
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle = shuffle, drop_last = drop_last, num_workers = num_workers)
    return dataloader


#with open("the-verdict.txt" , 'r') as file:
#    data = file.read()      Run this to get the data


dataloader = create_dataloader(data, batch_size=1, max_len=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

# [tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])] ----->> it gives input_tensor and output_tensor

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [18]:
# Creatiing Token Embedding

input_ids = torch.tensor([2,3,5,1])

In [19]:
vocab_size = 50257
output_dim = 256

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim) # Here number of rows is equal to vocab_size

print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035,  ...,  1.3337,  0.0771, -0.0522],
        [ 0.2386,  0.1411, -1.3354,  ..., -0.0315, -1.0640,  0.9417],
        [-1.3152, -0.0677, -0.1350,  ..., -0.3181, -1.3936,  0.5226],
        ...,
        [ 0.5871, -0.0572, -1.1628,  ..., -0.6887, -0.7364,  0.4479],
        [ 0.4438,  0.7411,  1.1263,  ...,  1.2091,  0.6781,  0.3331],
        [-0.2537,  0.1446,  0.7203,  ..., -0.2134,  0.2144,  0.3006]],
       requires_grad=True)


In [20]:
max_len = 4
dataloader = create_dataloader(data, batch_size=8, max_len=max_len, stride=max_len, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token Id : \n", inputs)
print("Input Shape : ", inputs.shape)

Token Id : 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Input Shape :  torch.Size([8, 4])


In [21]:
token_embedding = embedding_layer(inputs)
print(token_embedding.shape) # This will give 3 dimensional matrix which formed by embedding vector for all the 8*4 so output is ------>> 8*4*256

torch.Size([8, 4, 256])


In [22]:
# Create another embedding layer for positional encoding
# Here the number of row is context_length
context_length = max_len
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim) # This creates the embedding layer for positional  encoding

pos_embedding  = pos_embedding_layer(torch.arange(max_len))
print(pos_embedding.shape)

# This encodes the data to positional encoding to form size of 4*256
# Why we need to encode to get 4*256? ----  Because we are encoding positional encode and we want to know which position does input present
# So we get 4*256 

torch.Size([4, 256])


In [23]:
input_embedding = token_embedding + pos_embedding
print(input_embedding.shape)

# As total embedding of positional embedding is addidtion of token embedding and unquie positional embedding

torch.Size([8, 4, 256])


In [24]:
# Implementing simple attention mechanism
import torch
input_sam = torch.tensor(
    [[0.43, 0.15, 0.89],   #your   (x^1)
    [0.55, 0.87, 0.66],    #journey (x^2)
    [0.57, 0.85, 0.64],    #starts  (x^3)
    [0.22, 0.58, 0.33],    #with  (x^4)
    [0.77, 0.25, 0.10],    #one (x^5)
    [0.05, 0.80, 0.55]])   #step (x^6)

In [25]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# # corresponding words

# words_sam = ['Your','journey', 'starts', 'with', 'one', 'step']

# # Extract x, y, z coordinates
# x_coords = input_sam[: , 0].numpy()
# y_coords = input_sam[: , 1].numpy()
# z_coords = input_sam[: , 2].numpy()

# # creating 3D plot
# fig = plt.Figure()
# ax = fig.add_subplot(111, projection='3d')

# for x,y,z,word in zip(x_coords,y_coords, z_coords, words_sam):
#     ax.scatter(x,y,z)
#     ax.text(x,y,z,word, fontsize = 10)

# ax.set_xlabel("x")
# ax.set_ylabel("y")
# ax.set_zlabel("z")

# plt.title("3D plot of word Embedding")
# plt.show()

In [26]:
# Attention Score with only 2nd element
query = input_sam[1]
attn_score_2 = torch.empty(input_sam.shape[0])
for i,x_i in enumerate(input_sam):
    attn_score_2[i] = torch.dot(x_i,query) # dot product (transpose not necessary)

print(attn_score_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [27]:
# normalization of attention score

attn_weight_2_tmp = attn_score_2/attn_score_2.sum()

print("Attention weights : ", attn_weight_2_tmp)
print("Sum : ", attn_weight_2_tmp.sum())

Attention weights :  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum :  tensor(1.0000)


In [28]:
# Using Navie Softmax for normalization

def softmax_naive(x):
    return torch.exp(x)/torch.exp(x).sum(dim=0)

attn_weight_2_naive = softmax_naive(attn_score_2)

print("Attention weights : ", attn_weight_2_naive)
print("Sum : ", attn_weight_2_naive.sum())

Attention weights :  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum :  tensor(1.)


In [29]:
# Pytorch implementation of softmax

attn_weight_2 = torch.softmax(attn_score_2, dim=0) # Attention weights
print("Attention weights : ", attn_weight_2)
print("Sum : ", attn_weight_2.sum())

# Naive softmax may encounter numerical instability problems such as overflow and underflow
# It is always advisable to use Pytorch softmax

Attention weights :  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum :  tensor(1.)


In [30]:
# context vector implementation
query = input_sam[1]

context_vector = torch.zeros(query.shape)

for i, x_i in enumerate(input_sam):
    context_vector += attn_weight_2[i]*x_i

print(context_vector)

tensor([0.4419, 0.6515, 0.5683])


In [31]:
# Efficient way for attention score

attn_score = input_sam @ input_sam.T # It does dot product by multiplying input*input(transpose)
print(attn_score)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [32]:
attn_weight = torch.softmax(attn_score, dim=-1) # Attention weight for all the elements
print(attn_weight)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [33]:
all_context_vectors = attn_weight @ input_sam # All elements context vectors
print(all_context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [34]:
# self attention mechanism with query key and value

x_2 = input_sam[1]
d_in = input_sam.shape[1]
d_out = 2

torch.manual_seed(123)
w_query = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
w_key = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
w_value = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)

print(w_query,"\n")
print(w_key,"\n")
print(w_value,"\n")

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]]) 

Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]]) 

Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]]) 



In [35]:
# Query Key Value for input_sam[1]

query_2 = x_2 @ w_query
key_2 = x_2 @ w_key
value_2 = x_2 @ w_value

print(query_2,"\n")
print(key_2,"\n")
print(value_2,"\n")


tensor([0.4306, 1.4551]) 

tensor([0.4433, 1.1419]) 

tensor([0.3951, 1.0037]) 



In [36]:
# Query Key Value using input_sam

query= input_sam @ w_query
key = input_sam @ w_key
value = input_sam @ w_value

print(query,"\n")
print(key,"\n")
print(value,"\n")

tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2983, 0.6565],
        [0.2568, 1.0533]]) 

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]]) 

tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]]) 



In [37]:
key_2 = key[1]
attn_score_22 = query_2.dot(key_2)
print("Attention score 22 \n",attn_score_22,"\n")

attn_score_2 = query_2 @ key.T # All attention score for given query   Here query is input_sam[2]
print("Attention score 2 \n",attn_score_2,"\n")

attn_score = query @ key.T # omega
print("Attention score \n", attn_score)

Attention score 22 
 tensor(1.8524) 

Attention score 2 
 tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440]) 

Attention score 
 tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])


In [38]:
d_k = key.shape[-1]
attn_weights_2 = torch.softmax(attn_score_2/ d_k**0.5, dim=-1)
print(attn_weights_2)
print(d_k) 

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
2


In [39]:
# sample context vector with weights, values and keys
context_vec_2 = attn_weight_2 @ value 
print(context_vec_2)

tensor([0.3069, 0.8188])


In [40]:
# Modular self attention implementation for full input

import torch.nn as nn


class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.w_query = nn.Parameter(torch.rand(d_in, d_out))
        self.w_key = nn.Parameter(torch.rand(d_in, d_out))
        self.w_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self,x):
        key = x @ self.w_key    
        query = x @ self.w_query
        value = x @ self.w_value

        attn_scores = query @ key.T
        attn_weights = torch.softmax(attn_scores / key.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ value
        return context_vec
    
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)       # This is the context vector of input_sam
print(sa_v1(input_sam))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [41]:
# Enhanced version of self attention 

import torch.nn as nn

class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self,x):
        key =  self.w_key(x)    
        query = self.w_query(x)
        value = self.w_value(x) 

        attn_scores = query @ key.T
        attn_weights = torch.softmax(attn_scores / key.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ value
        return context_vec
    
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)       # This is the context vector of input_sam in an enhanced way
print(sa_v2(input_sam)) 

# version 1 and version 2 are completely different because they both use different initialization
# version 1 ---> nn.Parameter        version 2 ---> nn.Linear

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [42]:
queries = sa_v2.w_query(input_sam)
keys = sa_v2.w_key(input_sam)
attention_score = queries @ keys.T
attention_weights = torch.softmax(attention_score/keys.shape[-1]**0.5, dim=1)
print(attention_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [45]:
context_length = attention_score.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length)) # tril gives lower triangular matrix    triu gives upper triangular matrix
print(mask_simple,"\n")

masked_simple = attention_weights * mask_simple
print(masked_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]]) 

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [46]:
# normalizing masked_simple to make the sum of rows is equal to 1
row_sum = masked_simple.sum(dim=1, keepdim=True)
masked_simple_normalize = masked_simple/row_sum
print(masked_simple_normalize)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [ ]:
# Efficient masking
mask = torch.triu(torch.ones(context_length, context_length),diagonal=1)
masked = attention_score.masked_fill(mask.bool(), -torch.inf)
print("Efficiently Masked \n",masked, "\n")  # It gives masked matrix

attention_weights_eff = torch.softmax(masked/key.shape[-1]**0.5,dim=1)
print("Efficiennt attention weight after masked \n", attention_weights_eff) # Softmax after masked will give us normalized attention weight matrix

Efficiently Masked 
 tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>) 

Efficiennt attention weight after masked 
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
# Dropout for reducing thw lazy neurons 
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
# example = torch.ones(6,6)  example
# print(dropout(example))
print(dropout(attention_weights_eff))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


In [61]:
# Implementation of causal mechanism 

class CausalAttention(nn.Module):
    def __init__(self,d_in, d_out, context_length, dropout, qkv_bias = False):
        super().__init__()
        self.d_out = d_out
        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length),diagonal=1))

    def forward(self, x):
        b, num_token, d_in = x.shape
        key =  self.w_key(x)    
        query = self.w_query(x)
        value = self.w_value(x) 

        attn_scores = query @ key.transpose(1,2)
        attn_score.masked_fill(self.mask.bool() [:num_token, :num_token], -torch.inf)
        attn_weights = torch.softmax(attn_scores / key.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        context_vector_causal = attn_weights @ value
        return context_vector_causal

batch = torch.stack((input_sam, input_sam), dim=0) # shape of batch is [2,6,3]
torch.manual_seed(123)
context_length = batch.shape[1]
causalAttention = CausalAttention(d_in, d_out, context_length, 0.0)
context_vec_causal = causalAttention(batch)
print("Shape of context vector for causal attention \n", context_vec_causal.shape)



Shape of context vector for causal attention 
 torch.Size([2, 6, 2])


In [65]:
# Implementation Multi head attentioon mechanism

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_head, qkv_bias = False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in,d_out,context_length,dropout,qkv_bias)
             for _ in range (num_head)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)
    
torch.manual_seed(123)
batch = torch.stack((input_sam, input_sam), dim=0) 
context_length = batch.shape[1]
d_in, d_out = 3, 2
multihead = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_head=2)
context_vec_multihead = multihead(batch)
print("Context Vector of Multi head Attention \n", context_vec_multihead, "\n")
print(context_vec_multihead.shape)

Context Vector of Multi head Attention 
 tensor([[[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>) 

torch.Size([2, 6, 4])


In [66]:
# Implementation of Multi head attention mechanism with weight split

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_head, qkv_bias = False):
        super().__init__()
        assert(d_out % num_head == 0),\
            "d_out must be divisible by num_head"
        self.d_out = d_out
        self.num_head = num_head
        self.head_dim = d_out//num_head # Reduce the production dim to match the desire output dim
        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_in,d_out) # linear projection
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length),diagonal=1))
        
    def forward(self, x):
        b, num_token, d_in = x.shape
        key =  self.w_key(x)    
        query = self.w_query(x)
        value = self.w_value(x) 

        # We implicity split the matrix by adding a num_head dimension 
        # unroll last dim : (b, num_token, d_out) -> (b, num_token, num_head, head_dim)
        key = key.view(b, num_token, self.num_head, self.head_dim)
        value = value.view(b, num_token, self.num_head, self.head_dim)
        query = query.view(b, num_token, self.num_head, self.head_dim)

        # Transpose: (b, num_token, num_head, head_dim) -> (b, num_head, num_token, head_dim)
        key = key.transpose(1,2)
        query = query.transpose(1,2)
        value = value.transpose(1,2)

        # Compute scaled dot product attention aka self attention with causal mask
        attention_score = query @ key.transpose(2,3)

        # original mask truncated to the number of tokens and convert to boolean 
        mask_bool = self.mask.bool()[:num_token, :num_token]

        attention_score.masked_fill(mask_bool, -torch.inf) # masking the attention score

        attention_weight = torch.softmax(attention_score/key.shape[-1]**0.5, dim = -1)
        attention_weight = self.dropout(attention_weight)

        context_vector_multihead = (attention_weight @ value).transpose(1,2)

        # Combine head, where self.d_out = self.num_head * self.head_dim
        context_vector_multihead = context_vector_multihead.contiguous().view(b, num_token, self.d_out)
        context_vector_multihead = self.out_proj(context_vector_multihead) # Optional projection

        return context_vector_multihead
    
torch.manual_seed(123)

inputs = torch.tensor(
    [
    [0.43, 0.15, 0.89, 0.55, 0.87, 0.66],    
    [0.57, 0.85, 0.64, 0.22, 0.58, 0.33],    
    [0.77, 0.25, 0.10, 0.05, 0.80, 0.55]
    ])
batch = torch.stack((inputs, inputs), dim=0)
batch_size, context_length, d_in = batch.shape
d_out = 6
multiheadAttentionComplete = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_head=2)
context_vector = multiheadAttentionComplete(batch)
print("Context Vector for Multihead Attention \n", context_vector,"\n")
print("Context Vector shape \n", context_vector.shape, "\n")

Context Vector for Multihead Attention 
 tensor([[[ 0.1195, -0.0484,  0.0306, -0.0639, -0.2782, -0.2564],
         [ 0.1208, -0.0497,  0.0319, -0.0638, -0.2779, -0.2566],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]],

        [[ 0.1195, -0.0484,  0.0306, -0.0639, -0.2782, -0.2564],
         [ 0.1208, -0.0497,  0.0319, -0.0638, -0.2779, -0.2566],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]]],
       grad_fn=<ViewBackward0>) 

Context Vector shape 
 torch.Size([2, 3, 6]) 

